In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
os.listdir('data')

['Env_QH.csv',
 'micro_sud3.pkl',
 'AllPM_QH.csv',
 'micro_sud3_normalized.pkl',
 'AllNO2_QH.csv']

In [24]:
df = pd.read_pickle('data/micro_sud3_normalized.pkl')
df = df.reset_index()

def split_dataframe(dataframe, percent):
    nb_rows = int(np.floor(percent * len(dataframe)))
    return dataframe[:nb_rows], dataframe[nb_rows:]

def dataframe_to_xy(df, look_back):
    i = look_back
    while True:
        sequence = df.iloc[i - look_back:i]
        yield np.array(sequence[['NO2_61FD', 'NO2_61F0', 'NO2_61EF', 'temp', 'rh',\
                                 'tgrad', 'pressure', 'pluvio']]).reshape(look_back, 1, 8) , np.array(sequence['NO2_ref'])
        i += 1
        if i == len(df):
            i = look_back
            
def dataframe_to_xy_test(df, look_back):
    X_test, y_test = [], []
    i = look_back
    while i < len(df):
        sequence = df.iloc[i - look_back:i]
        X_test.append(np.array(sequence[['NO2_61FD', 'NO2_61F0', 'NO2_61EF', 'temp', 'rh',\
                                 'tgrad', 'pressure', 'pluvio']]).reshape(look_back, 1, 8))
        y_test.append(np.array(sequence['NO2_ref']))
        i += 1
        if i == len(df):
            break
            
    return np.array(X_test), np.array(y_test)
            
        
df_train, df_test = split_dataframe(df, 0.5) 
df_valid, df_test = split_dataframe(df_test, 0.5)

X_train = dataframe_to_xy(df_train, 10)
X_valid = dataframe_to_xy(df_valid, 10)
X_test, y_test = dataframe_to_xy_test(df_test, 10)

In [4]:
a = next(X_train)

In [14]:
from keras.layers import SimpleRNN, Dense, LSTM, GRU
from keras.models import Sequential
from keras.callbacks import EarlyStopping

def simple_rnn_model(nb_units, input_dim, loss='mean_squared_error', optimizer='adam'):
    model = Sequential()
    model.add(SimpleRNN(nb_units, input_shape=input_dim))#input_dim=input_dim[1], input_length=input_dim[0], return_sequences=True))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model

def lstm_model(nb_units, input_dim, loss='mean_squared_error', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(nb_units, input_shape=input_dim))#input_dim=input_dim[1], input_length=input_dim[0], return_sequences=True))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model

def gru_model(nb_units, input_dim, loss='mean_squared_error', optimizer='adam'):
    model = Sequential()
    model.add(GRU(nb_units, input_shape=(input_dim)))#input_dim=input_dim[1], input_length=input_dim[0], return_sequences=True))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss=loss, optimizer=optimizer)
    model.summary()
    return model

In [15]:
model = simple_rnn_model(16, (1, 8))
history = model.fit_generator(X_train, epochs=100, steps_per_epoch=32, validation_data=X_valid, validation_steps=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 16)                400       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 417
Trainable params: 417
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
32/32 [==============================] - 0s - loss: 475.5028 - val_loss: 442.5375
Epoch 2/100
32/32 [==============================] - 0s - loss: 3632.3933 - val_loss: 843.8432
Epoch 3/100
32/32 [==============================] - 0s - loss: 2233.3465 - val_loss: 3537.1894
Epoch 4/100
32/32 [==============================] - 0s - loss: 2589.7701 - val_loss: 1791.1738
Epoch 5/100
32/32 [==============================] - 0s - loss: 1919.5568 - val_loss: 1072.5757
Epoch 6/100
32/32 [==============================] - 0s - loss:

32/32 [==============================] - 0s - loss: 239.0018 - val_loss: 5029.8900
Epoch 81/100
32/32 [==============================] - 0s - loss: 993.0958 - val_loss: 1190.1211
Epoch 82/100
32/32 [==============================] - 0s - loss: 2415.3060 - val_loss: 160.3148
Epoch 83/100
32/32 [==============================] - 0s - loss: 3689.4667 - val_loss: 877.4035
Epoch 84/100
32/32 [==============================] - 0s - loss: 3447.2473 - val_loss: 244.0852
Epoch 85/100
32/32 [==============================] - 0s - loss: 298.9517 - val_loss: 399.1573
Epoch 86/100
32/32 [==============================] - 0s - loss: 1995.0718 - val_loss: 4031.9966
Epoch 87/100
32/32 [==============================] - 0s - loss: 2534.2621 - val_loss: 129.5962
Epoch 88/100
32/32 [==============================] - 0s - loss: 814.5319 - val_loss: 1988.5288
Epoch 89/100
32/32 [==============================] - 0s - loss: 4687.3632 - val_loss: 373.3134
Epoch 90/100
32/32 [==============================] -

In [28]:
y_pred = model.predict(X_test)
plt.plot(history.history['loss'], 'r-')
plt.plot(history.history['val_loss'], 'b-')
plt.show()

plt.title('y_pred, y_test')
y_pred
plt.plot(y_pred[:, 0], y_pred[:, 1], '+')
plt.show()

ValueError: Error when checking : expected simple_rnn_2_input to have 3 dimensions, but got array with shape (554, 10, 1, 8)

In [27]:
y_pred.shape

(100, 1)